# Extract Frames from Video

In [ ]:
#
##
import torchvision
#
## define video to extract frames from
var_path_video = "../dataset/video/act_49_41.mp4"
var_name = var_path_video.split("/")[-1].split(".")[0]
#
## define frames to extract
var_frame_index = list(range(0, 90, 1))
print(var_name, var_frame_index)

In [ ]:
#
## read video
data_frame, _, var_info = torchvision.io.read_video(var_path_video)
data_frame.shape

In [ ]:
#
##
for var_i in var_frame_index:
    #
    data_image = data_frame[var_i].permute(2, 0, 1)
    #
    torchvision.io.write_jpeg(data_image, var_name + "_" + str(var_i) + ".jpg", 100)


# Visualize WiFi CSI

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as scio
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer

In [ ]:
#
##
var_path_mat = "../dataset/wifi_csi/mat/act_49_41.mat"
data_mat = scio.loadmat(var_path_mat)

In [ ]:
#
##
var_length = data_mat["trace"].shape[0]
#
## 
data_csi_amp = []
for var_t in range(var_length):
    data_csi_amp.append(abs(data_mat["trace"][var_t][0][0][0][-1]))
#
data_csi_amp = np.array(data_csi_amp, dtype = np.float32)
#
print(data_csi_amp.shape)

In [ ]:
#
##
data_csi_show = abs(data_csi_amp.reshape(data_csi_amp.shape[0], -1))
print(data_csi_show.shape)
#
##
data_csi_show = np.transpose(data_csi_show)
var_x_encoder = MinMaxScaler() 
data_csi_show = var_x_encoder.fit_transform(data_csi_show)
data_csi_show = np.transpose(data_csi_show)
#
##
var_f, _ = plt.subplots(1, 1)
plt.axis("off")
var_image = plt.imshow(data_csi_show.T, interpolation = "none", aspect = "auto", cmap = "viridis")

In [ ]:
var_name = var_path_mat.split("/")[-1].split(".")[0]
var_f.savefig(var_name + ".png", bbox_inches = "tight", pad_inches = 0, dpi = 600)

# Dataset Statistic

In [1]:
#
##
import pandas as pd
#
data_pd_y = pd.read_csv("../dataset/annotation.csv")

In [6]:
var_column = ["user_1_activity", "user_2_activity", "user_3_activity", "user_4_activity", "user_5_activity", "user_6_activity"]
for var_c in var_column:
    print(data_pd_y[var_c].value_counts())

nothing     528
walk        528
rotation    528
jump        528
wave        528
lie_down    528
pick_up     528
sit_down    528
stand_up    528
Name: user_1_activity, dtype: int64
walk        528
rotation    528
jump        528
wave        528
lie_down    528
pick_up     528
sit_down    528
stand_up    528
nothing     528
Name: user_2_activity, dtype: int64
rotation    528
jump        528
wave        528
lie_down    528
pick_up     528
sit_down    528
stand_up    528
nothing     528
walk        528
Name: user_3_activity, dtype: int64
jump        528
wave        528
lie_down    528
pick_up     528
sit_down    528
stand_up    528
nothing     528
walk        528
rotation    528
Name: user_4_activity, dtype: int64
wave        528
lie_down    528
pick_up     528
sit_down    528
stand_up    528
nothing     528
walk        528
rotation    528
jump        528
Name: user_5_activity, dtype: int64
nothing     528
walk        528
rotation    528
jump        528
wave        528
lie_down    528
pick

In [9]:
var_column = ["user_1_location", "user_2_location", "user_3_location", "user_4_location", "user_5_location", "user_6_location"]
for var_c in var_column:
    print(data_pd_y[var_c].value_counts().sort_index())

a    1296
b     864
c     864
d     864
e     864
Name: user_1_location, dtype: int64
a     864
b    1296
c     864
d     864
e     864
Name: user_2_location, dtype: int64
a     864
b     864
c    1296
d     864
e     864
Name: user_3_location, dtype: int64
a     864
b     864
c     864
d    1296
e     864
Name: user_4_location, dtype: int64
a     864
b     864
c     864
d     864
e    1296
Name: user_5_location, dtype: int64
a    1134
b    1026
c     864
d     864
e     864
Name: user_6_location, dtype: int64


## Packet Loss

In [57]:
#
##
import os
import numpy as np
import pandas as pd
#
##
def load_data_y(var_path_data_y,
                var_environment = None, 
                var_wifi_band = None, 
                var_num_users = None):
    #
    ##
    data_pd_y = pd.read_csv(var_path_data_y, dtype = str)
    #
    if var_environment is not None:
        data_pd_y = data_pd_y[data_pd_y["environment"].isin(var_environment)]
    #
    if var_wifi_band is not None:
        data_pd_y = data_pd_y[data_pd_y["wifi_band"].isin(var_wifi_band)]
    #
    if var_num_users is not None:
        data_pd_y = data_pd_y[data_pd_y["number_of_users"].isin(var_num_users)]
    #
    return data_pd_y

#
##
def summarize_packet_loss(var_dir_amp, data_pd_y):
    #
    ##
    var_length_x = []
    #
    for var_c, var_label in enumerate(list(data_pd_y["label"])):
        #
        data_x = np.load(os.path.join(var_dir_amp, var_label + ".npy"))
        #
        var_length_x.append(data_x.shape[0])
    #
    var_length_x = np.array(var_length_x)
    print(np.average(var_length_x))
    #
    var_loss_rate = (3000 - var_length_x) / 3000 * 100
    var_avg = np.average(var_loss_rate)
    var_std = np.std(var_loss_rate)
    #
    return var_avg, var_std

In [58]:
#
##
data_pd_y = load_data_y("../dataset/annotation.csv", var_wifi_band = ["2.4"])
var_path_x = "../dataset/wifi_csi/amp"
var_avg, var_std = summarize_packet_loss(var_path_x, data_pd_y)
#
print("Avg: %.2f - Std: %.2f "%(var_avg, var_std))

2864.393762183236
Avg: 4.52 - Std: 2.75 


In [59]:
#
##
data_pd_y = load_data_y("../dataset/annotation.csv", var_wifi_band = ["5"])
var_path_x = "../dataset/wifi_csi/amp"
var_avg, var_std = summarize_packet_loss(var_path_x, data_pd_y)
#
print("Avg: %.2f - Std: %.2f "%(var_avg, var_std))

2930.7210703526493
Avg: 2.31 - Std: 1.57 
